In [1]:
from genetic_algorithm import *
import re
import json
import numpy as np
from collections import Counter
import copy

## hexadecimal generator

In [2]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 2
nr_of_genes = 16
nr_individuals = 4

individuals = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

In [3]:
## sum duplicates
individuals_sum_dup = sum_duplicated_neurons(individuals)

## remove self loops
individuals_sum_dup_no_self_loop = remove_self_loop(individuals_sum_dup)

## brain generator
# one individual to change to multiple
for individual in individuals_sum_dup_no_self_loop:
#     try:
    edges = individuals_sum_dup_no_self_loop[individual]
    edges = [tuple(edges[i]) for i in edges]

    ## preprocessing
    dic = weight_sum_preprocessing(edges)

    ## calculate weight sum
#     except:
#         print(individual)
#         print(edges)
        
#         continue
    try:
        mid_to_weight(dic)
    except:
        print('mid_to_weight', individual)
        print(edges)
        print()
        continue   
    try:    
        out_to_weight(dic)
    except:
        print('out_to_weight', individual)
        print(edges)
        print()    

out_to_weight 1
[('in1', 'out0', 1.4279871814436136), ('in0', 'out0', -3.0119945063329774), ('mid1', 'out0', -1.1555012971158247), ('in1', 'mid1', -1.2089729894704715), ('mid1', 'out2', -2.6342743781474134), ('mid0', 'out0', -3.9923088661681674)]



In [10]:
individuals_sum_dup_no_self_loop[1]

{'1in0out': ['in1', 'out0', 1.4279871814436136],
 '0in0out': ['in0', 'out0', -3.0119945063329774],
 '1mid0out': ['mid1', 'out0', -1.1555012971158247],
 '1in1mid': ['in1', 'mid1', -1.2089729894704715],
 '1mid2out': ['mid1', 'out2', -2.6342743781474134],
 '0mid0out': ['mid0', 'out0', -3.9923088661681674]}

In [8]:
edges = [('in1', 'out0', 1.4279871814436136), ('in0', 'out0', -3.0119945063329774), ('mid1', 'out0', -1.1555012971158247), ('in1', 'mid1', -1.2089729894704715), ('mid1', 'out2', -2.6342743781474134), ('mid0', 'out0', -3.9923088661681674)]

In [9]:
dic = weight_sum_preprocessing(edges)
dic

{'mid1': {'in1': -1.2089729894704715},
 'out0': {'in1': 1.4279871814436136,
  'in0': -3.0119945063329774,
  'mid1': -1.1555012971158247,
  'mid0': -3.9923088661681674},
 'out2': {'mid1': -2.6342743781474134}}

In [11]:
def mid_to_weight(dic):
    '''weight for clear input ie. no nested weights: 0mid:{0in, 1in} or 0mid:{0mid, 0in}'''
    for key in dic:
        
        if 'mid' in key and isinstance(dic[key], dict):
            print(key)
            if set(i[:2] for i in dic[key]) == set(['in']):
                
                dic[key] = np.tanh(sum(dic[key].values()))
                mid_to_weight(dic)
                
            elif key in dic[key].keys() and Counter(i[:2] for i in dic[key])['mi'] == 1:
                
                print(dic[key])
                dic[key] = np.tanh(sum(dic[key].values()))
                mid_to_weight(dic)
mid_to_weight(dic)
dic

mid1


{'mid1': -0.83637115452817,
 'out0': {'in1': 1.4279871814436136,
  'in0': -3.0119945063329774,
  'mid1': -1.1555012971158247,
  'mid0': -3.9923088661681674},
 'out2': {'mid1': -2.6342743781474134}}

In [7]:
def out_to_weight(dic):
    '''weight for nested input ie.  0mid:{1mid} or out:{0mid, 0in}'''
    for key in dic:
        if 'mid' in key and isinstance(dic[key], dict):
            for sub_key in dic[key]:
                if 'mid' in sub_key and sub_key != key:
                    dic[key][sub_key] += dic[sub_key]

            dic[key] = np.tanh(sum(dic[key].values()))
            out_to_weight(dic)
                
        elif 'out' in key and isinstance(dic[key], dict):
            for sub_key in dic[key]:
                if 'mid' in sub_key:
                    dic[key][sub_key] += dic[sub_key]
            dic[key] = np.tanh(sum(dic[key].values()))
            out_to_weight(dic)
            
out_to_weight(dic)

TypeError: unsupported operand type(s) for +=: 'float' and 'dict'

In [ ]:
mid_to_weight(dic)
dic

In [ ]:
dic